In [2]:
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding
from keras.models import Sequential
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import time

%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set()

import warnings
warnings.filterwarnings('ignore')


In [3]:
df = pd.read_csv('../FeatureEngineering/MetaData/data6_&_odds.csv')
df.dropna(inplace=True)

In [4]:
train_data = df.loc[(df.season <= 2013) & (df.season >= 2007)]
valid_data = df.loc[(df.season > 2013) & (df.season < 2016)]
test_data = df.loc[df.season >= 2016]
full_train_data = pd.concat([train_data, valid_data], axis=0)

X, y = train_data.drop(columns=['home_team_wins']), train_data.home_team_wins
valid_X, valid_y = valid_data.drop(columns=['home_team_wins']), valid_data.home_team_wins
test_X, test_y = test_data.drop(columns=['home_team_wins']), test_data.home_team_wins

# Split our data
X_train, y_train = train_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), train_data.home_team_wins
X_val, y_val = valid_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), valid_data.home_team_wins
X_test, y_test = test_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), test_data.home_team_wins


In [5]:
arr_X_train = X_train.to_numpy()
arr_X_train = arr_X_train.reshape(arr_X_train.shape[0], arr_X_train.shape[1], 1)

arr_X_val = X_val.to_numpy()
arr_X_val = arr_X_val.reshape(arr_X_val.shape[0], arr_X_val.shape[1], 1)



### Making a LSTM model

### 3X sigmoid

In [17]:
model = Sequential()
model.add(LSTM(90, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(90, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])




In [48]:
# model = get_model(input_shape=(100,1))

In [ ]:
model.fit(arr_X_train, y_train, epochs=100, batch_size=32)


In [10]:
# evaluate the keras model
scores = model.evaluate(arr_X_val, y_val, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 68.12


We are testing different parameters for the LSTM model

### Tanh, relu, sigmoid

In [19]:
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
model.fit(arr_X_train, y_train, epochs=20, batch_size=64)


In [21]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 129, 100)          40800     
                                                                 
 dropout_8 (Dropout)         (None, 129, 100)          0         
                                                                 
 lstm_9 (LSTM)               (None, 129, 100)          80400     
                                                                 
 dropout_9 (Dropout)         (None, 129, 100)          0         
                                                                 
 dense_4 (Dense)             (None, 129, 1)            101       
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
# evaluate the keras model
scores = model.evaluate(arr_X_val, y_val, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 68.81


### Recurrent dropout

In [32]:
model = Sequential()
model.add(LSTM(50, activation='tanh', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
model.fit(arr_X_train, y_train, epochs=50, batch_size=32)
print(model.summary())

In [34]:
# evaluate the keras model
scores = model.evaluate(arr_X_val, y_val, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 68.90


### Tanh, relu, sigmoid with 100 epochs

In [36]:
model = Sequential()
model.add(LSTM(50, activation='tanh', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
model.fit(arr_X_train, y_train, epochs=100, batch_size=32)


In [38]:
# evaluate the keras model
scores = model.evaluate(arr_X_val, y_val, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 69.69


## Test the models

In [6]:
train_data = df.loc[(df.season < 2016) & (df.season >= 2007)]
test_data = df.loc[df.season >= 2016]

X, y = train_data.drop(columns=['home_team_wins']), train_data.home_team_wins
test_X, test_y = test_data.drop(columns=['home_team_wins']), test_data.home_team_wins

# Split our data
X_train, y_train = train_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), train_data.home_team_wins
X_test, y_test = test_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), test_data.home_team_wins


In [ ]:
arr_X_test = X_test.to_numpy()
arr_X_test = arr_X_test.reshape(arr_X_test.shape[0], arr_X_test.shape[1], 1)

### 3X Sigmoid

In [ ]:
model = Sequential()
model.add(LSTM(90, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(90, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

In [7]:
model.fit(arr_X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
216/216 [==============================] - 76s 294ms/step - loss: 0.6730 - accuracy: 0.6025
Epoch 2/100
216/216 [==============================] - 66s 306ms/step - loss: 0.6612 - accuracy: 0.6204
Epoch 3/100
216/216 [==============================] - 66s 307ms/step - loss: 0.6578 - accuracy: 0.6253
Epoch 4/100
216/216 [==============================] - 64s 296ms/step - loss: 0.6537 - accuracy: 0.6278
Epoch 5/100
216/216 [==============================] - 62s 287ms/step - loss: 0.6529 - accuracy: 0.6290
Epoch 6/100
216/216 [==============================] - 63s 292ms/step - loss: 0.6500 - accuracy: 0.6292
Epoch 7/100
216/216 [==============================] - 65s 299ms/step - loss: 0.6502 - accuracy: 0.6288
Epoch 8/100
216/216 [==============================] - 67s 310ms/step - loss: 0.6460 - accuracy: 0.6313
Epoch 9/100
216/216 [==============================] - 66s 304ms/step - loss: 0.6456 - accuracy: 0.6316
Epoch 10/100
216/216 [==============================] - 64s 296m

In [ ]:
# evaluate the keras model
scores = model.evaluate(arr_X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 65.53


### Tanh, relu, sigmoid

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
model.fit(arr_X_train, y_train, epochs=20, batch_size=64)


In [ ]:
# evaluate the keras model
scores = model.evaluate(arr_X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 65.95


### Recurrent dropout

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='tanh', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
model.fit(arr_X_train, y_train, epochs=50, batch_size=32)


In [ ]:
# evaluate the keras model
scores = model.evaluate(arr_X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 66.72


### Tanh, relu, sigmoid with 100 epochs

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='tanh', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu', recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [ ]:
model.fit(arr_X_train, y_train, epochs=100, batch_size=32)


In [ ]:
# evaluate the keras model
scores = model.evaluate(arr_X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (scores[1]*100))

Accuracy: 66.98
